In [1]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import sys
import holidays


import sklearn
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


import xgboost
from xgboost import XGBRegressor
import catboost
from catboost import CatBoostRegressor

In [2]:
print(f"파이썬 버전 : {sys.version}")
print(f"pandas 버전 : {pd.__version__}")
print(f"numpy 버전 : {np.__version__}")
print(f"sklearn 버전 : {sklearn.__version__}")
print(f"xgboost 버전 확인 : {xgboost.__version__}")
print(f"catboost 버전 : {catboost.__version__}")

파이썬 버전 : 3.10.13 (main, Mar 21 2024, 13:39:43) [GCC 11.4.0]
pandas 버전 : 2.2.1
numpy 버전 : 1.26.4
sklearn 버전 : 1.4.1.post1
xgboost 버전 확인 : 2.0.3
catboost 버전 : 1.2.3


In [21]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2024) # Seed 고정

In [7]:
train = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/train.csv')
test = pd.read_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/test.csv')

In [5]:
def pre_all(train, test):
    print(f"전처리 전 train 크기 : {train.shape}")
    print(f"전처리 전 test 크기 : {test.shape}")
    print("=================전처리 중=================")

    # 합쳐서 전처리하기
    train["timestamp"] = pd.to_datetime(train["timestamp"])
    test["timestamp"] = pd.to_datetime(test["timestamp"])
    df = pd.concat([train,test]).reset_index(drop = True)

    df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'},inplace=True)

    #년/월/일 추가
    df['year']=df['timestamp'].dt.year
    df['month']=df['timestamp'].dt.month
    df['day']=df['timestamp'].dt.day

    #요일 추가
    df['week_day']=df['timestamp'].dt.weekday

    # 년-월 변수 추가 : year-month의 형태, 개월단위 누적값
    le = LabelEncoder()
    df["year_month"] = df["timestamp"].map(lambda x :str(x.year) + "-"+str(x.month))

    # 라벨 인코딩
    df["year_month"] = le.fit_transform(df["year_month"])


    # 주차 변수 추가
    df["week"] = df["timestamp"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

    # 주차 누적값
    week_list=[]
    for i in range(len(df['year'])) :
        if df['year'][i] == 2019 :
            week_list.append(int(df['week'][i]))
        elif df['year'][i] == 2020 :
            week_list.append(int(df['week'][i])+52)
        elif df['year'][i] == 2021 :
            week_list.append(int(df['week'][i])+52+53)
        elif df['year'][i] == 2022 :
            week_list.append(int(df['week'][i])+52+53+53)
        elif df['year'][i] == 2023 :
            week_list.append(int(df['week'][i])+52+53+53+52)
    df['week_num']= week_list

    # datetime 패키지에서 19년 12월 마지막주가 첫째주로 들어가는거 발견하여 수정
    df.loc[df['timestamp']=='2019-12-30','week_num']=52
    df.loc[df['timestamp']=='2019-12-31','week_num']=52


    # 공휴일 변수 추가
    def make_holi(x):
        kr_holi = holidays.KR()

        if x in kr_holi:
            return 1
        else:
            return 0
        
    df["holiday"] = df["timestamp"].map(lambda x : make_holi(x))
    

    # train, test 분리하기
    train = df[~df["price"].isnull()].sort_values("timestamp").reset_index(drop = True)
    test = df[df["price"].isnull()].sort_values("timestamp").reset_index(drop=True)


    print(f"전처리 후 train 크기 : {train.shape}")
    print(f"전처리 후 test 크기 : {test.shape}")

    return train, test

In [8]:
train_pre, test_pre = pre_all(train, test)

전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)


In [9]:
train_pre

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,1,1,1
1,CB_A_S_20190101,2019-01-01,CB,A,S,0.0,0.0,2019,1,1,1,0,1,1,1
2,RD_D_J_20190101,2019-01-01,RD,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
3,BC_D_J_20190101,2019-01-01,BC,D,J,0.0,0.0,2019,1,1,1,0,1,1,1
4,CB_F_J_20190101,2019-01-01,CB,F,J,0.0,0.0,2019,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,CR_E_S_20230303,2023-03-03,CR,E,S,0.0,0.0,2023,3,3,4,50,9,219,0
59393,BC_A_S_20230303,2023-03-03,BC,A,S,3776.0,2875.0,2023,3,3,4,50,9,219,0
59394,CB_E_J_20230303,2023-03-03,CB,E,J,0.0,0.0,2023,3,3,4,50,9,219,0
59395,BC_D_J_20230303,2023-03-03,BC,D,J,1776.0,3059.0,2023,3,3,4,50,9,219,0


In [18]:
test_pre

,ID,timestamp,item,corporation,location,supply,price,year,month,day,week_day,year_month,week,week_num,holiday
0,TG_A_J_20230304,2023-03-04,TG,A,J,NaN,NaN,2023,3,4,5,50,9,219,0
1,TG_E_S_20230304,2023-03-04,TG,E,S,NaN,NaN,2023,3,4,5,50,9,219,0
2,BC_B_J_20230304,2023-03-04,BC,B,J,NaN,NaN,2023,3,4,5,50,9,219,0
3,TG_E_J_20230304,2023-03-04,TG,E,J,NaN,NaN,2023,3,4,5,50,9,219,0
4,BC_B_S_20230304,2023-03-04,BC,B,S,NaN,NaN,2023,3,4,5,50,9,219,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,TG_A_J_20230331,2023-03-31,TG,A,J,NaN,NaN,2023,3,31,4,50,13,223,0
1088,RD_D_J_20230331,2023-03-31,RD,D,J,NaN,NaN,2023,3,31,4,50,13,223,0
1089,CR_D_J_20230331,2023-03-31,CR,D,J,NaN,NaN,2023,3,31,4,50,13,223,0
1090,TG_E_J_20230331,2023-03-31,TG,E,J,NaN,NaN,2023,3,31,4,50,13,223,0


In [19]:
## 전처리

# 극 이상치 제거
# tg_idx = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]>20000)].index
tg_idx = train_pre[(train_pre["item"]=="CR") & (train_pre["price"]>20000)].index
rd_idx = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]>5000)].index
bc_idx = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]>8000)].index
cb_idx = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]>2300)].index

# train_pre.loc[tg_idx,"price"] = train_pre[(train_pre["item"]=="TG") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[tg_idx,"price"] = train_pre[(train_pre["item"]=="CR") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[rd_idx,"price"] = train_pre[(train_pre["item"]=="RD") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[bc_idx,"price"] = train_pre[(train_pre["item"]=="BC") & (train_pre["price"]!=0)]["price"].mean()
train_pre.loc[cb_idx,"price"] = train_pre[(train_pre["item"]=="CB") & (train_pre["price"]!=0)]["price"].mean()


# 감귤이 아닌것
print(f"train의 컬럼 : {train_pre.columns}")
print(f"test의 컬럼 : {test_pre.columns}")

train_notg = train_pre[train_pre["item"] !="TG"]
test_notg = test_pre[test_pre["item"] != "TG"]

train의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')
test의 컬럼 : Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'day', 'week_day', 'year_month', 'week', 'week_num',
       'holiday'],
      dtype='object')


In [20]:
#인코딩
Xy = pd.get_dummies(train_notg.sort_values(by = ["timestamp"]).reset_index(drop=True).drop(columns = ["supply"]), columns = ["item","corporation","location"])
answer_notg = pd.get_dummies(test_notg.drop(columns = ["timestamp","supply","price"]), columns = [ "item","corporation","location"])
print(Xy.columns)

Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_BC', 'item_CB',
       'item_CR', 'item_RD', 'corporation_A', 'corporation_B', 'corporation_C',
       'corporation_D', 'corporation_E', 'corporation_F', 'location_J',
       'location_S'],
      dtype='object')


In [22]:
Xy.drop(columns = ["timestamp", "ID","price"])

,year,month,day,week_day,year_month,week,week_num,holiday,item_BC,item_CB,item_CR,item_RD,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2019,1,1,1,0,1,1,1,False,True,False,False,True,False,False,False,False,False,False,True
1,2019,1,1,1,0,1,1,1,False,False,True,False,False,False,False,True,False,False,False,True
2,2019,1,1,1,0,1,1,1,False,False,False,True,False,False,False,True,False,False,False,True
3,2019,1,1,1,0,1,1,1,False,False,True,False,False,False,False,False,True,False,False,True
4,2019,1,1,1,0,1,1,1,False,False,False,True,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44162,2023,3,3,4,50,9,219,0,False,True,False,False,False,False,False,True,False,False,True,False
44163,2023,3,3,4,50,9,219,0,False,True,False,False,True,False,False,False,False,False,False,True
44164,2023,3,3,4,50,9,219,0,True,False,False,False,False,False,False,True,False,False,True,False
44165,2023,3,3,4,50,9,219,0,False,False,True,False,False,False,True,False,False,False,True,False


In [23]:
## 앙상블 모델 정의

cat = CatBoostRegressor(random_state = 2024, 
                            n_estimators = 1000, 
                            learning_rate = 0.01, 
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_notg.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_notg["answer"] = pred

answer_notg[["ID","answer"]]

0:	learn: 934.3461033	total: 52.1ms	remaining: 52.1s
999:	learn: 376.6995081	total: 3.98s	remaining: 0us


,ID,answer
2,BC_B_J_20230304,2384.968678
4,BC_B_S_20230304,100.733655
6,BC_C_J_20230304,2293.269558
7,BC_A_S_20230304,2751.574472
10,BC_D_J_20230304,2776.219702
...,...,...
1085,RD_D_S_20230331,457.576344
1086,CR_C_J_20230331,1783.102829
1088,RD_D_J_20230331,398.828636
1089,CR_D_J_20230331,1822.411070


# 3. TG (1)
### 3-1. 전처리

In [24]:
train_pre, test_pre = pre_all(train, test)

# 공휴일이지만 안쉬는 날 제외하기
no_holi = list(train_pre[(train_pre["item"] =="TG") &(train_pre["holiday"]==1) & (train_pre["price"]!=0)].groupby("timestamp").count().reset_index()["timestamp"])
noholi_idx = train_pre[train_pre["timestamp"].isin(no_holi)]["holiday"].index
for idx in noholi_idx:
    train_pre.loc[idx, "holiday"] = 0

# train 및 test 시간 순서로 정렬하기
train_tg = train_pre[train_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)
test_tg = test_pre[test_pre["item"] == "TG"].sort_values(by = ["timestamp"]).reset_index(drop= True)

Xy = pd.get_dummies(train_tg, columns = [ "item","corporation","location"]).drop(columns = ["supply"])
answer_tg1 = pd.get_dummies(test_tg, columns = [ "item","corporation","location"]).drop(columns = ["timestamp","supply","price"])
print(f"train의 컬럼 : {Xy.columns}")
print(f"test의 컬럼 : {answer_tg1.columns}")
Xy["price"] = np.sqrt(Xy["price"])

전처리 전 train 크기 : (59397, 7)
전처리 전 test 크기 : (1092, 5)
=================전처리 중=================
전처리 후 train 크기 : (59397, 15)
전처리 후 test 크기 : (1092, 15)
train의 컬럼 : Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'item_TG', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')
test의 컬럼 : Index(['ID', 'year', 'month', 'day', 'week_day', 'year_month', 'week',
       'week_num', 'holiday', 'item_TG', 'corporation_A', 'corporation_B',
       'corporation_C', 'corporation_D', 'corporation_E', 'location_J',
       'location_S'],
      dtype='object')


### 3-2. 모델링 & 훈련 예측

In [25]:
# # 모델 정의
cat = CatBoostRegressor(random_state = 2024, 
                            n_estimators = 1000, 
                            learning_rate = 0.01, 
                            depth = 10,
                            l2_leaf_reg = 3,
                            metric_period = 1000)

xgb = XGBRegressor(n_estimators = 1000, random_state = 2024, learning_rate = 0.01, max_depth = 10)


# voting
vote_model = VotingRegressor(
    estimators =[("cat",cat), ("xgb", xgb)]
)

vote_model.fit(Xy.drop(columns = ["timestamp", "ID","price"]), Xy["price"])

pred = vote_model.predict(answer_tg1.drop(columns = ["ID"]))
for idx in range(len(pred)):
    if pred[idx]<0:
        pred[idx]= 0
answer_tg1["answer"] = np.power(pred,2)

answer_tg1[["ID","answer"]]

0:	learn: 30.9664058	total: 3.96ms	remaining: 3.96s
999:	learn: 15.5181153	total: 2.93s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2563.261796
1,TG_E_S_20230304,3458.001764
2,TG_E_J_20230304,377.066719
3,TG_D_S_20230304,3662.783068
4,TG_D_J_20230304,607.878918
...,...,...
275,TG_D_J_20230331,2804.801999
276,TG_D_S_20230331,4860.908623
277,TG_A_S_20230331,5304.802147
278,TG_E_S_20230331,4859.787923


# 4. TG (2)
- 일반화를 위한 추가 모델링
### 4-1. 전처리

In [26]:
train_tg2 = train_pre[train_pre["item"] =="TG"]
test_tg2 = test_pre[test_pre["item"] == "TG"]

Xy2 = pd.get_dummies(train_tg2.sort_values(by = ["timestamp", "corporation","location"]).reset_index(drop=True).drop(columns = ["item","supply"]), columns = [ "corporation","location"])
answer_tg2 = pd.get_dummies(test_tg2.drop(columns = ["timestamp","supply","price","item"]), columns = [ "corporation","location"])
print(Xy2.columns)

# 종속변수 루트값
Xy2["price"] = np.sqrt(Xy2["price"])

Index(['ID', 'timestamp', 'price', 'year', 'month', 'day', 'week_day',
       'year_month', 'week', 'week_num', 'holiday', 'corporation_A',
       'corporation_B', 'corporation_C', 'corporation_D', 'corporation_E',
       'location_J', 'location_S'],
      dtype='object')


### 4-2. 모델링 & 훈련 예측

In [27]:
# 모델 정의 및 훈련 예측
n_estimators =1000
lrs = 0.05
max_depths = 10
l2_leaf_reg = 3

cat = CatBoostRegressor(random_state = 2024, 
                                n_estimators = n_estimators, 
                                learning_rate = lrs, 
                                depth = max_depths, 
                                l2_leaf_reg = l2_leaf_reg,
                                metric_period = 1000)

cat.fit(Xy2.drop(columns = ["timestamp", "ID","price"]), Xy2["price"])

pred2 = cat.predict(answer_tg2.drop(columns = ["ID"]))
for idx in range(len(pred2)):
    if pred2[idx]<0:
        pred2[idx]= 0
answer_tg2["answer"] = np.power(pred2,2)

answer_tg2[["ID","answer"]]

0:	learn: 30.2823698	total: 3.48ms	remaining: 3.48s
999:	learn: 11.0655757	total: 3.03s	remaining: 0us


,ID,answer
0,TG_A_J_20230304,2742.172780
1,TG_E_S_20230304,3354.003496
3,TG_E_J_20230304,760.581106
5,TG_D_S_20230304,3769.615969
8,TG_D_J_20230304,53.451592
...,...,...
1074,TG_D_S_20230331,4727.895634
1077,TG_A_S_20230331,5403.967583
1079,TG_E_S_20230331,4715.201735
1087,TG_A_J_20230331,6813.935154


# 5. TG 앙상블

In [28]:
total1 = pd.concat([answer_tg1[["ID","answer"]],answer_notg[["ID","answer"]]])
total2 = pd.concat([answer_tg2[["ID","answer"]],answer_notg[["ID","answer"]]])

In [32]:
print(total1.shape)
print(total2.shape)

(1092, 2)
(1092, 2)


In [29]:
# TG 앙상블 (평균)

df = pd.merge(total1, total2, how = "inner", on="ID")

In [31]:
df

,ID,answer_x,answer_y
0,TG_A_J_20230304,2563.261796,2742.172780
1,TG_E_S_20230304,3458.001764,3354.003496
2,TG_E_J_20230304,377.066719,760.581106
3,TG_D_S_20230304,3662.783068,3769.615969
4,TG_D_J_20230304,607.878918,53.451592
...,...,...,...
1087,RD_D_S_20230331,457.576344,457.576344
1088,CR_C_J_20230331,1783.102829,1783.102829
1089,RD_D_J_20230331,398.828636,398.828636
1090,CR_D_J_20230331,1822.411070,1822.411070


In [33]:

df["answer"] = (df["answer_x"]+df["answer_y"])/2
df["item"] = df["ID"].map(lambda x :x.split("_")[0])

In [6]:
# 후처리 
## 전체 min값 | 3월의 min값 확인
df.loc[(df['item']=='TG')&(df['answer']<400),'answer'] =0 # 551   #3월 675
df.loc[(df['item']=='CB')&(df['answer']<50),'answer'] =0 # 162  # 3월 200
df.loc[(df['item']=='RD')&(df['answer']<10),'answer'] =0 # 50     # 3월 124
df.loc[(df['item']=='CR')&(df['answer']<150),'answer'] =0 # 250   # 3월 450
df.loc[(df['item']=='BC')&(df['answer']<100),'answer'] =0 #205 3월 205.0

In [10]:
df = df.drop(columns = ["answer_x","answer_y", "item"])
df

           ID          ds       y
0      TG_A_J  2019-01-01     0.0
1      TG_A_J  2019-01-02     0.0
2      TG_A_J  2019-01-03  1728.0
3      TG_A_J  2019-01-04  1408.0
4      TG_A_J  2019-01-05  1250.0
...       ...         ...     ...
59392  RD_F_J  2023-02-27   468.0
59393  RD_F_J  2023-02-28   531.0
59394  RD_F_J  2023-03-01   574.0
59395  RD_F_J  2023-03-02   523.0
59396  RD_F_J  2023-03-03   529.0

[59397 rows x 3 columns]


In [11]:
# 제출용
df.to_csv('/mnt/c/Users/wschu/OneDrive/Documents/data/jeju_specialty/open/answer.csv',index=False)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params
-----------------------------------------------------
0 | criterion       | MSELoss          | 0     
1 | train_metrics   | MetricCollection | 0     
2 | val_metrics     | MetricCollection | 0     
3 | decomposition   | _SeriesDecomp    | 0     
4 | linear_seasonal | Linear           | 3.6 K 
5 | linear_trend    | Linear           | 3.6 K 
-----------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 31.09it/s, train_loss=5.13e+6]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 30.62it/s, train_loss=5.13e+6]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 31.87it/s, train_loss=5.69e+4]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 31.56it/s, train_loss=5.69e+4]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 256.85it/s]
